In [1]:
from PIL import Image
from lang_sam import LangSAM
import supervision as sv
import numpy as np

import matplotlib.pyplot as plt
import torch
from pathlib import Path
import os

model = LangSAM()

/Users/rolandmeertens/anaconda3/envs/langsegment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/rolandmeertens/anaconda3/envs/langsegment/lib/python3.10/site-packages/groundingdino/models/GroundingDINO/ms_deform_attn.py:31: UserWarning: Failed to load custom C++ ops. Running on CPU mode Only!
  warnings.warn("Failed to load custom C++ ops. Running on CPU mode Only!")
/Users/rolandmeertens/anaconda3/envs/langsegment/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3527.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased
Model loaded from /Users/rolandmeertens/.cache/huggingface/hub/models--ShilongLiu--GroundingDINO/snapshots/a94c9b567a2a374598f05c584e96798a170c56fb/groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])


In [2]:
mask_annotator = sv.MaskAnnotator()

def get_masks_from_image(image_pil):
    all_boxes = list()
    all_masks = list()
    all_classes = list()

    #for index, text_prompt in enumerate(["pedestrian", "car", "van", 'traffic light']): 
    #for index, text_prompt in enumerate(["ambulance"]): 
    for index, text_prompt in enumerate(["stroller"]): 
        with torch.no_grad(): 
            masks, boxes, phrases, logits = model.predict(image_pil, text_prompt)

        if len(masks) > 0: 
            all_boxes.append(boxes.numpy())
            all_masks.append(masks.numpy())
            all_classes.extend([index for _ in range(len(boxes))])

    if len(all_boxes) > 0: 
        boxes = np.vstack(all_boxes)
        masks = np.vstack(all_masks)
    else: 
        boxes = np.zeros((4, 0))
        masks = np.zeros((4, 0))
    all_classes = np.array(all_classes)
    return boxes, masks, all_classes

In [3]:
scene_folder = Path('/Users/rolandmeertens/datasets/driveSafeAI/v0/scene_1/')
rgb_folder = scene_folder / 'rgb_undistorted'
semseg_result_folder = scene_folder / 'semseg_undistorted' / 'wayve_sam_visualised'

for i, image in enumerate(sorted(rgb_folder.glob('*_ff.jpeg'))): 
    image_pil = Image.open(image).convert("RGB")
    print(image.name)
    b, m, classes = get_masks_from_image(image_pil)

    if b.size > 0: 
        d = sv.Detections(xyxy=b, mask=m, class_id=classes)
        annotated_image = mask_annotator.annotate(np.array(image_pil), d)
    else: 
        annotated_image = np.array(image_pil)

    plt.imsave(os.path.join('scene 1 LangSAM stroller', image.name), annotated_image)

1625498661219143_ff.jpeg


/Users/rolandmeertens/anaconda3/envs/langsegment/lib/python3.10/site-packages/transformers/modeling_utils.py:907: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/Users/rolandmeertens/anaconda3/envs/langsegment/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/Users/rolandmeertens/anaconda3/envs/langsegment/lib/python3.10/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


1625498661266262_ff.jpeg
1625498661305241_ff.jpeg
1625498661346292_ff.jpeg
1625498661388563_ff.jpeg
1625498661428703_ff.jpeg
1625498661464067_ff.jpeg
1625498661505081_ff.jpeg
1625498661547761_ff.jpeg
1625498661581761_ff.jpeg
1625498661623376_ff.jpeg
1625498661661639_ff.jpeg
1625498661701004_ff.jpeg
1625498661744566_ff.jpeg
1625498661777762_ff.jpeg
1625498661827549_ff.jpeg
1625498661868633_ff.jpeg
1625498661904682_ff.jpeg
1625498661946252_ff.jpeg
1625498661980518_ff.jpeg
1625498662029285_ff.jpeg
1625498662057996_ff.jpeg
1625498662112589_ff.jpeg
1625498662144789_ff.jpeg
1625498662184573_ff.jpeg
1625498662221318_ff.jpeg
1625498662265648_ff.jpeg
1625498662308284_ff.jpeg
1625498662346090_ff.jpeg
1625498662384015_ff.jpeg
1625498662426214_ff.jpeg
1625498662465872_ff.jpeg
1625498662507305_ff.jpeg
1625498662543116_ff.jpeg
1625498662583844_ff.jpeg
1625498662624961_ff.jpeg
1625498662667000_ff.jpeg
1625498662704576_ff.jpeg
1625498662748400_ff.jpeg
1625498662786156_ff.jpeg
1625498662823822_ff.jpeg
